In [106]:
import pandas as pd 
d = pd.read_csv('/Users/qitianhu/Documents/Research/Explore/COVID-19_sosc/Data/276城_3source_by_ct_V3.1.csv')
d['xunshi'] = d.prov.apply(lambda x: 1 if x in ['福建省', '河南省', '四川省', '贵州省', '辽宁省', '黑龙江省', '江苏省', '山东省', '湖南省', '宁夏回族自治区', '广东省', '海南省', '河北省', '山西省'] else 0)
d['tenure_bi'] = d.tenure.apply(lambda a: int(a >= 3))
d.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'ct_shortname', 'prov', 'citycode',
       'ctnm', 'centlon', 'centlat', 'locked_down', 'lockdown_date',
       'bdidx_19m20', 'xc_lockdown', 'xc_closed', 'daySinceFirstCase',
       'sub_prov_ct', 'gdp2018', '自治州-盟-地区', 'in_291', 'pdensity', 'gdp_p',
       'hospital_d', 'popHR18_all', 'Log_popHR18_all', 'gdp_per_10k',
       'primary_ind', 'second_ind', 'third_ind', 'prov_leader_rank',
       'num_hospital_total', 'num_doctors_total', 'num_firm_total',
       'non_domestic_firms_total', 'pct_of_non_domestic_firm',
       'primary_emp_share_total', 'secondary_emp_share_total',
       'tertiary_emp_share_total', 'inaug_time', 'birthmonth', 'is_female',
       'age_feb20', 'party_age', 'work_age', 'tenure', 'majorchara',
       'is_STEM_major', 'rule_in_native_prov', 'is_BA', 'is_MA', 'is_PhD',
       'cumulative_case', 'log_cumulative_case', 'bdidx_19m20_feb1_10',
       'lockdown_datenum', 'xc_lockdown_datenum', 'xc_closed_datenum',
       'hu_liao

In [ ]:
d.groupby('xunshi').mean()['tenure'].sort_values()

xunshi
1    1.566265
0    2.438776
Name: tenure, dtype: float64

In [4]:
# 看第十一轮和第十二轮的影响
d['xunshi0'] = d.prov.apply(lambda x: 1 if x in ['甘肃省', '广西壮族自治区', '云南省', '陕西省'] else 0)
d.groupby('xunshi0').mean()['tenure'].sort_values()

xunshi0
0    1.696429
1    2.975000
Name: tenure, dtype: float64

## IV assumptions

In [13]:
import statsmodels.api as sm
import numpy as np 

model = sm.OLS(d.tenure, d[['xunshi' ,'gdp_per_10k', 'prov_leader_rank']])
res = model.fit()
res.t_test([1, 0])

ValueError: wrong shape for coefs

import statsmodels.api as sm
import numpy as np

<!-- duncan_prestige = sm.datasets.get_rdataset("Duncan", "carData")
Y = duncan_prestige.data['income']
X = duncan_prestige.data['education'] -->
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.params
const        10.603498
education     0.594859
dtype: float64

# select good vars

In [14]:
# import json 
# json.load
# with open('/Users/qitianhu/Documents/Research/Explore/COVID-19_sosc/STAT274/sensitivity_analysis/var_groups.json', 'r') as json_file:
#     dct = json.load(json_file)
dct = {
    "lockdown":["locked_down", "lockdown_date", "bdidx_19m20", "xc_lockdown", "xc_closed"],
    "economic": ["sub_prov_ct", "gdp2018",
        "pdensity", "gdp_p", "hospital_d", "popHR18_all", "Log_popHR18_all",
        "gdp_per_10k", "primary_ind", "second_ind", "third_ind",
        "prov_leader_rank", "num_hospital_total", "num_doctors_total","pct_of_non_domestic_firm", "primary_emp_share_total",
        "secondary_emp_share_total", "tertiary_emp_share_total"],
    "personal": ["inaug_time",
        "birthmonth", "is_female", "age_feb20", "party_age", "work_age",
        "tenure", "majorchara", "is_STEM_major", "rule_in_native_prov", "is_BA",
        "is_MA", "is_PhD"],
    "covid": ["cumulative_case", "log_cumulative_case"]
}



In [15]:
import statsmodels.api as sm
import numpy as np



In [16]:
[c1,c2,c3]

['sub_prov_ct', 'gdp2018', 'in_291']

### pure linear regression coef selection

In [62]:
dct1 = dct['economic']
done_pair = []
# s = set()
for c1i in range(len(dct1)):
    for c2i in range(c1i, len(dct1)):
        for c3i in range(c2i, len(dct1)):
            c1,c2,c3 = dct1[c1i], dct1[c2i],dct1[c3i]
            if c1 != c2 and c2!= c3 and c3!= c1:
                mod = sm.OLS(d['xc_lockdown'], d[['tenure']+[c1,c2,c3]] )
                res = mod.fit()
                if float(res.f_test('tenure = 0').summary().split(',')[1][3:]) < .01:
                    print([c1, c2, c3])
                # s.add(set([c1,c2,c3]))
                


['sub_prov_ct', 'gdp2018', 'pdensity']
['sub_prov_ct', 'gdp2018', 'gdp_p']
['sub_prov_ct', 'gdp2018', 'hospital_d']
['sub_prov_ct', 'gdp2018', 'gdp_per_10k']
['sub_prov_ct', 'gdp2018', 'primary_ind']
['sub_prov_ct', 'gdp2018', 'prov_leader_rank']
['sub_prov_ct', 'gdp2018', 'num_hospital_total']
['sub_prov_ct', 'gdp2018', 'num_doctors_total']
['sub_prov_ct', 'gdp2018', 'pct_of_non_domestic_firm']
['sub_prov_ct', 'gdp2018', 'primary_emp_share_total']
['sub_prov_ct', 'pdensity', 'gdp_p']
['sub_prov_ct', 'pdensity', 'hospital_d']
['sub_prov_ct', 'pdensity', 'gdp_per_10k']
['sub_prov_ct', 'pdensity', 'primary_ind']
['sub_prov_ct', 'pdensity', 'prov_leader_rank']
['sub_prov_ct', 'pdensity', 'num_hospital_total']
['sub_prov_ct', 'pdensity', 'num_doctors_total']
['sub_prov_ct', 'pdensity', 'pct_of_non_domestic_firm']
['sub_prov_ct', 'pdensity', 'primary_emp_share_total']
['sub_prov_ct', 'gdp_p', 'hospital_d']
['sub_prov_ct', 'gdp_p', 'gdp_per_10k']
['sub_prov_ct', 'gdp_p', 'prov_leader_rank']


In [42]:
res = mod.fit()
res.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Results: Ordinary least squares
======================================================================
Model:                 OLS               Adj. R-squared:      -0.006  
Dependent Variable:    xc_lockdown       AIC:                 376.5809
Date:                  2021-12-05 15:13  BIC:                 390.8847
No. Observations:      264               Log-Likelihood:      -184.29 
Df Model:              3                 F-statistic:         0.5192  
Df Residuals:          260               Prob (F-statistic):  0.669   
R-squared:             0.006             Scale:               0.24016 
----------------------------------------------------------------------
                          Coef.  Std.Err.   t    P>|t|   [0.025 0.975]
----------------------------------------------------------------------
tenure                    0.0069   0.0200 0.3455 0.7300 -0.0324 0.0462
tertiary_emp_share_total  0.0025   0.0012 2.0312 0.0433  0.0001 0.0050
secondary_emp_share_total 0.0053   0.0013 4.2172 0.0000  0.0028 0.0078
primary_emp_share_total   0.0036   0.0045 0.7906 0.4299 -0.0053 0.0124
----------------------------------------------------------------------
Omnibus:               1591.697        Durbin-Watson:           2.110 
Prob(Omnibus):         0.000           Jarque-Bera (JB):        43.461
Skew:                  0.447           Prob(JB):                0.000 
Kurtosis:              1.225           Condition No.:           46    
======================================================================

"""

In [56]:
# res.params
float(res.f_test('tenure = 0').summary().split(',')[1][3:]) < .05

False

### IV selection 

In [113]:
from statsmodels.sandbox.regression.gmm import IV2SLS
# endog is the dependent variable, y
# exog is the x matrix that has the endogenous information in it. Include the endogenous variables in it.
# instrument is the z matrix. Include all the variables that are not endogenous and replace the endogenous variables from the exog matrix (above) with what ever instruments you choose for them.


# no_endog_results = IV2SLS(endog, exog_constant, instrument = instr_constant).fit()
dct1 = dct['economic'] + ['log_cumulative_case']
done_pair = []
# s = set()
for c1i in range(len(dct1)):
    for c2i in range(c1i, len(dct1)):
        for c3i in range(c2i, len(dct1)):
            c1,c2,c3 = dct1[c1i], dct1[c2i], dct1[c3i]
            if c1 != c2 and c2!= c3 and c3!= c1:
                res = IV2SLS(d['xc_lockdown'], d[['tenure_bi']+[c1,c2,c3]], instrument= d[['xunshi']+[c1,c2,c3]]).fit() 
                if float(res.f_test('tenure_bi = 0').summary().split(',')[1][3:]) < .001:
                    print([c1, c2, c3])
                # s.add(set([c1,c2,c3]))
                


['sub_prov_ct', 'gdp2018', 'gdp_per_10k']
['sub_prov_ct', 'gdp2018', 'pct_of_non_domestic_firm']
['sub_prov_ct', 'gdp2018', 'primary_emp_share_total']
['sub_prov_ct', 'gdp_per_10k', 'pct_of_non_domestic_firm']
['sub_prov_ct', 'gdp_per_10k', 'primary_emp_share_total']
['gdp2018', 'gdp_per_10k', 'pct_of_non_domestic_firm']
['gdp2018', 'gdp_per_10k', 'primary_emp_share_total']


In [110]:
possible_pairs =[['sub_prov_ct', 'gdp2018', 'gdp_per_10k'],
    ['sub_prov_ct', 'gdp2018', 'pct_of_non_domestic_firm'],
    ['sub_prov_ct', 'gdp2018', 'primary_emp_share_total'],
    ['sub_prov_ct', 'gdp_per_10k', 'pct_of_non_domestic_firm'],
    ['sub_prov_ct', 'gdp_per_10k', 'primary_emp_share_total'],
    ['sub_prov_ct', 'pct_of_non_domestic_firm', 'primary_emp_share_total'],
    ['gdp2018', 'gdp_per_10k', 'pct_of_non_domestic_firm'],
    ['gdp2018', 'gdp_per_10k', 'primary_emp_share_total'],
    ['gdp2018', 'pct_of_non_domestic_firm', 'primary_emp_share_total'],
    ['gdp_per_10k', 'pct_of_non_domestic_firm', 'primary_emp_share_total']]


for pair in possible_pairs:
    res = IV2SLS(d['xc_lockdown'], d[['tenure_bi']+pair], 
        instrument= d[['xunshi']+pair]).fit() 
    print(pair, res.params['tenure_bi'])
    


['sub_prov_ct', 'gdp2018', 'gdp_per_10k'] 1.3325741463438256
['sub_prov_ct', 'gdp2018', 'pct_of_non_domestic_firm'] 1.0572005456292333
['sub_prov_ct', 'gdp2018', 'primary_emp_share_total'] 1.3454640967668023
['sub_prov_ct', 'gdp_per_10k', 'pct_of_non_domestic_firm'] 1.041462150002561
['sub_prov_ct', 'gdp_per_10k', 'primary_emp_share_total'] 1.374792644294527
['sub_prov_ct', 'pct_of_non_domestic_firm', 'primary_emp_share_total'] 1.0266520004343076
['gdp2018', 'gdp_per_10k', 'pct_of_non_domestic_firm'] 0.9765090020444052
['gdp2018', 'gdp_per_10k', 'primary_emp_share_total'] 1.2962766614144055
['gdp2018', 'pct_of_non_domestic_firm', 'primary_emp_share_total'] 1.0055648808052962
['gdp_per_10k', 'pct_of_non_domestic_firm', 'primary_emp_share_total'] 1.015057498181365


In [112]:
pair = ['sub_prov_ct', 'gdp_per_10k', 'primary_emp_share_total']

res = IV2SLS(d['xc_lockdown'], d[['tenure_bi']+pair], instrument= d[['xunshi']+pair]).fit() 
print(res.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}             &   xc\_lockdown   & \textbf{  R-squared:         } &    -0.629   \\
\textbf{Model:}                     &      IV2SLS      & \textbf{  Adj. R-squared:    } &    -0.654   \\
\textbf{Method:}                    &    Two Stage     & \textbf{  F-statistic:       } &       nan   \\
\textbf{}                           &  Least Squares   & \textbf{  Prob (F-statistic):} &      nan    \\
\textbf{Date:}                      & Sun, 05 Dec 2021 & \textbf{                     } &             \\
\textbf{Time:}                      &     16:29:13     & \textbf{                     } &             \\
\textbf{No. Observations:}          &         264      & \textbf{                     } &             \\
\textbf{Df Residuals:}              &         260      & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
                                    & \textbf{coef} & \textbf{s

In [79]:
res.f_test('tenure = 0')

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[-1.80136348e-17]]), p=1.0, df_denom=260, df_num=1>

In [114]:
res = IV2SLS(d['xc_lockdown'], d[['tenure_bi']+pair], instrument= d[['xunshi']+pair]).fit() 
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:            xc_lockdown   R-squared:                      -0.629
Model:                         IV2SLS   Adj. R-squared:                 -0.654
Method:                     Two Stage   F-statistic:                       nan
                        Least Squares   Prob (F-statistic):                nan
Date:                Sun, 05 Dec 2021                                         
Time:                        23:02:10                                         
No. Observations:                 264                                         
Df Residuals:                     260                                         
Df Model:                           4                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
tenure_bi                   1.3748      0.357      3.850      0.000       0.672       2.078
sub_prov_ct                 0.4611      0.276      1.669      0.096      -0.083       1.005
gdp_per_10k             -7.124e-07    1.2e-06     -0.593      0.554   -3.08e-06    1.65e-06
primary_emp_share_total    -0.0035      0.008     -0.411      0.681      -0.020       0.013
==============================================================================
Omnibus:                       21.368   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                9.877
Skew:                          -0.261   Prob(JB):                      0.00717
Kurtosis:                       2.209   Cond. No.                     3.30e+05
==============================================================================
"""

In [118]:
d['age_bi'] = (d.age_feb20 > d.age_feb20.median()).apply(int)
res = IV2SLS(d['xc_lockdown'], d[['age_bi']+pair], instrument= d[['xunshi']+pair]).fit() 
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:            xc_lockdown   R-squared:                      -0.254
Model:                         IV2SLS   Adj. R-squared:                 -0.273
Method:                     Two Stage   F-statistic:                       nan
                        Least Squares   Prob (F-statistic):                nan
Date:                Sun, 05 Dec 2021                                         
Time:                        23:03:29                                         
No. Observations:                 264                                         
Df Residuals:                     260                                         
Df Model:                           4                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
age_bi                      1.2273      0.280      4.388      0.000       0.677       1.778
sub_prov_ct                -0.1147      0.222     -0.517      0.606      -0.551       0.322
gdp_per_10k             -1.004e-06    1.1e-06     -0.913      0.362   -3.17e-06    1.16e-06
primary_emp_share_total     0.0071      0.006      1.113      0.267      -0.005       0.020
==============================================================================
Omnibus:                        5.726   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.057   Jarque-Bera (JB):                4.159
Skew:                          -0.175   Prob(JB):                        0.125
Kurtosis:                       2.495   Cond. No.                     3.28e+05
==============================================================================
"""

In [119]:
d.age_feb20.median()

56.0